In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
supply = pd.read_csv('stochastic_example_offer.csv').set_index('id')
nsupply = len(supply)
print(supply.shape)
supply.head()

(3, 4)


,node,capacity (MW),offer ($/MW),type
id,,,,
G1,Bus 1,100.0,10.0,inflexible
WP,Bus 1,70.0,0.0,wind
G2,Bus 2,30.0,20.0,flexible


In [3]:
demand = pd.read_csv('stochastic_example_bid.csv').set_index('id')
ndemand = len(demand)
ushed = demand['bid ($/MW)'].max()
print(demand.shape)
demand.head()

(1, 3)


,node,demand (MW),bid ($/MW)
id,,,
Load,Bus 2,120.0,80.0


In [4]:
scenarios = pd.read_csv('stochastic_example_scenarios.csv').set_index('scenario')
nscenarios = len(scenarios)
pi_w = 1. / nscenarios

# scenarios = scenarios.loc[[1]]
# nscenarios = 1
# pi_w = 1.

# scenarios = scenarios.loc[[]]
# nscenarios = 0
# pi_w = 0.

nscenarios = len(scenarios)
print(scenarios.shape)
scenarios.head()

(1, 2)


,id,actual (MW)
scenario,,
1,WP,5.0


In [5]:
nodes = sorted(set(supply['node']) | set(demand['node']))
nnodes = len(nodes)
print(nodes)

['Bus 1', 'Bus 2']


In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 5e3, 1e5] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
Bus 1-Bus 2,Bus 1,Bus 2,5000.0,100000.0


In [7]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns').fillna(0.)

,demand (MW),capacity (MW)
node,,
Bus 2,120.0,30.0
Bus 1,0.0,170.0


In [8]:
M = Model('power')

log = StringIO()
M.setLogHandler(log)
theta = M.variable('theta', nnodes, Domain.inRange(
    [0.] + [-np.pi] * (nnodes - 1), 
    [0.] + [np.pi] * (nnodes - 1), 
))
pD = M.variable('pD', ndemand, Domain.inRange(0., demand['demand (MW)'].values))
pG = M.variable('pG', nsupply, Domain.inRange(0., supply['capacity (MW)'].values))
UD = demand['bid ($/MW)'].values
CG = supply['offer ($/MW)'].values

line_constraints = Domain.inRange(-lines['capacity (MW)'].values, 
                                  lines['capacity (MW)'].values)
line_flows = M.variable('flows', nlines, line_constraints)
src_idx = [nodes.index(x) for x in lines.source]
dst_idx = [nodes.index(x) for x in lines.dest]
phase_diff = Expr.sub(theta.pick(src_idx), theta.pick(dst_idx))
phase_flow = M.constraint(Expr.sub(line_flows, 
                                  Expr.mulElm(lines['susceptance (S)'].values, phase_diff)),
                         Domain.equalsTo(0.))
node_balance_eqs = []
for node in nodes:
    supply_idx = np.flatnonzero(supply.node == node).astype('int32')
    demand_idx = np.flatnonzero(demand.node == node).astype('int32')
    node_supply = Expr.sum(pG.pick(supply_idx))
    node_demand = Expr.sum(pD.pick(demand_idx))
    balance = Expr.sub(node_demand, node_supply)
    direction = [1 if line.source == node else 
                 -1 if line.dest == node else 
                 0 for line_idx, line in lines.iterrows()]
    balance = Expr.add(balance, Expr.dot(direction, line_flows))
    balance_eq = M.constraint(node + 'balance', balance, Domain.equalsTo(0.))
    node_balance_eqs.append(balance_eq)

obj = Expr.sub(Expr.dot(UD, pD), Expr.dot(CG, pG))

M.constraint(pG.index(1), Domain.equalsTo(10.))

inflexible_idx = np.flatnonzero(supply.type == 'inflexible').astype('int32')
gdown = -supply['capacity (MW)'].values.copy()
gdown[inflexible_idx] = 0.
gup = supply['capacity (MW)'].values.copy()
gup[inflexible_idx] = 0.

wind_idx = np.flatnonzero(supply.type == 'wind').astype('int32')
nwind = len(wind_idx)
# flexible_idx = np.flatnonzero(supply.type == 'flexible')

node_balance_eqs_rt = []
for w_id, w in scenarios.groupby(level='scenario'):
    assert list(w.id) == list(supply.iloc[wind_idx].index)
    wind_actual = w['actual (MW)'].values
    
    theta_w = M.variable('theta%s' % w_id, nnodes, Domain.inRange(
        [0.] + [-np.pi] * (nnodes - 1), 
        [0.] + [np.pi] * (nnodes - 1), 
    ))
    pDrt = M.variable('pDrt%s' % w_id, ndemand, Domain.lessThan(0.))
    pGrt = M.variable('pGrt%s' % w_id, nsupply, Domain.inRange(gdown, gup))
    pGspill = M.variable('pGspill%s' % w_id, nwind, Domain.inRange(0., wind_actual))

    M.constraint(Expr.add(pD, pDrt), Domain.inRange(0., demand['demand (MW)'].values))
    M.constraint(Expr.add(pG, pGrt), Domain.inRange(0., supply['capacity (MW)'].values))
    
    total_wind = Expr.add(pG.pick(wind_idx), pGrt.pick(wind_idx))
    total_wind = Expr.add(total_wind, pGspill)
    M.constraint(total_wind, Domain.equalsTo(wind_actual))

    line_flows_w = M.variable('flows%s' % w_id, nlines, line_constraints)
    phase_diff_w = Expr.sub(theta_w.pick(src_idx), theta_w.pick(dst_idx))
    phase_flow_w = M.constraint(Expr.sub(line_flows_w, 
                                         Expr.mulElm(lines['susceptance (S)'].values, phase_diff_w)),
                                Domain.equalsTo(0.))
    node_balance_eqs_w = []
    for node in reversed(nodes):
        supply_idx = np.flatnonzero(supply.node == node).astype('int32')
        demand_idx = np.flatnonzero(demand.node == node).astype('int32')

        node_supply = Expr.sum(pG.pick(supply_idx))
        node_demand = Expr.sum(pD.pick(demand_idx))
        node_supply_w = Expr.sum(pGrt.pick(supply_idx))
        node_demand_w = Expr.sum(pDrt.pick(demand_idx))
        balance_w = Expr.sub(node_demand, node_supply)
        balance_w = Expr.add(balance_w, node_demand_w)
        balance_w = Expr.sub(balance_w, node_supply_w)
        direction = [1 if line.source == node else 
                     -1 if line.dest == node else 
                     0 for line_idx, line in lines.iterrows()]
        balance_w = Expr.add(balance_w, Expr.dot(direction, line_flows_w))
        balance_eq_w = M.constraint(node + 'balance%s' % w_id, balance_w, Domain.equalsTo(0.))
        node_balance_eqs_w.append(balance_eq_w)
    
    node_balance_eqs_rt.append(node_balance_eqs_w)
    
    obj_w = Expr.sub(Expr.mul(ushed, Expr.sum(pDrt)), Expr.dot(CG, pGrt))
    obj = Expr.add(obj, Expr.mul(pi_w, obj_w))

M.objective('obj', ObjectiveSense.Maximize, obj)
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal:
    M.writeTask('proj1.opf')

demand['DA'] = pD.level()
supply['DA'] = pG.level()
for w_id in scenarios.index.unique():
    demand['RT%s' % w_id] = pD.level() + M.getVariable('pDrt%s' % w_id).level()
    supply['RT%s' % w_id] = pG.level() + M.getVariable('pGrt%s' % w_id).level()
    
lines['flow (DA)'] = line_flows.level()

buses = pd.DataFrame({
    'volt. angle (rad)': theta.level(),
    'price (DA)': [e.dual()[0] for e in node_balance_eqs],
}, index=nodes)


In [9]:
with pd.option_context('display.float_format', '{:.1f}'.format):
    display(supply.head())

,node,capacity (MW),offer ($/MW),type,DA,RT1
id,,,,,,
G1,Bus 1,100.0,10.0,inflexible,100.0,100.0
WP,Bus 1,70.0,0.0,wind,10.0,5.0
G2,Bus 2,30.0,20.0,flexible,10.0,15.0


In [10]:
demand.head()

,node,demand (MW),bid ($/MW),DA,RT1
id,,,,,
Load,Bus 2,120.0,80.0,120.0,120.0


In [11]:
buses.head()

,volt. angle (rad),price (DA)
Bus 1,0.0000,-4.050149e-09
Bus 2,-0.0011,-4.050149e-09


In [12]:
lines

,source,dest,capacity (MW),susceptance (S),flow (DA)
id,,,,,
Bus 1-Bus 2,Bus 1,Bus 2,5000.0,100000.0,110.0


In [13]:
summary = pd.concat([
    buses['price (DA)'],
    demand.groupby('node')['DA'].sum(),
    supply.groupby('node')['DA'].sum(),
], axis=1).fillna(0.)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
Bus 1,-4.050149e-09,0.0,110.0
Bus 2,-4.050149e-09,120.0,10.0


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
Bus 1 & -4.050149e-09 &            0.0 &          110.0 \\
Bus 2 & -4.050149e-09 &          120.0 &           10.0 \\
\bottomrule
\end{tabular}



In [14]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(lines)
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

KeyError: 'consumed (MW)'

In [ ]:
print(log.getvalue())

In [ ]:
supply.query('`offer ($/MW)` <= 0.').groupby('node')[['capacity (MW)', 'supplied (MW)']].sum()

In [ ]:
supply.query('`offer ($/MW)` > 0.').groupby('node')[['capacity (MW)', 'supplied (MW)']].sum()

In [ ]:
demand.query('`bid ($/MW)` <= 0.').groupby('node')[['demand (MW)', 'consumed (MW)']].sum()

In [ ]:
demand.query('`bid ($/MW)` > 0.').groupby('node')[['demand (MW)', 'consumed (MW)']].sum()